# Day 08: Junction Box Circuit Analysis

This notebook analyzes junction boxes in 3D space and creates circuits by connecting nearby boxes.

In [21]:
# Setup: Add project root to path and import utilities
import sys
sys.path.insert(0, '..')

from utils import get_input

# Load input data for day 8
data = get_input(8)

print(f"Loaded {len(data)} lines of input")
print("Sample data:")
for line in data[:5]:
    print(f"  {line}")

Loaded 1000 lines of input
Sample data:
  27558,61383,12726
  15513,81970,25554
  24379,89821,82524
  42987,15460,38773
  10680,2978,15903


In [22]:
"""Core data classes for junction box circuit analysis."""
from dataclasses import dataclass, field
from typing import List, Optional, Iterator
from math import sqrt


@dataclass
class JunctionBox:
    """Represents a junction box in 3D space."""
    x: int
    y: int
    z: int
    circuit_id: Optional[int] = None
    
    def distance_to(self, other: 'JunctionBox') -> float:
        """Calculate Euclidean distance to another junction box."""
        return sqrt(
            (self.x - other.x) ** 2 + 
            (self.y - other.y) ** 2 + 
            (self.z - other.z) ** 2
        )
    
    @classmethod
    def from_string(cls, line: str) -> 'JunctionBox':
        """Parse a junction box from a comma-separated string."""
        x, y, z = map(int, line.split(','))
        return cls(x=x, y=y, z=z)
    
    def __repr__(self) -> str:
        circuit_info = f", circuit={self.circuit_id}" if self.circuit_id is not None else ""
        return f"JunctionBox({self.x}, {self.y}, {self.z}{circuit_info})"

In [23]:
@dataclass
class Connection:
    """Represents a connection between two junction boxes."""
    box1: JunctionBox
    box2: JunctionBox
    distance: float
    
    def __repr__(self) -> str:
        return f"Connection({self.box1} <-> {self.box2}, dist={self.distance:.2f})"
    
    @classmethod
    def from_boxes(cls, box1: JunctionBox, box2: JunctionBox) -> 'Connection':
        """Create a connection from two boxes, calculating distance."""
        return cls(box1, box2, box1.distance_to(box2))
    
    @classmethod
    def create_all_connections(cls, boxes: List[JunctionBox]) -> List['Connection']:
        """Create all possible connections between boxes."""
        return [
            cls.from_boxes(boxes[i], boxes[j])
            for i in range(len(boxes))
            for j in range(i + 1, len(boxes))
        ]

In [24]:
@dataclass
class Circuit:
    """Represents a circuit containing multiple junction boxes."""
    circuit_id: int
    _boxes: List[JunctionBox] = field(default_factory=list, repr=False)
    
    def __post_init__(self):
        """Assign circuit ID to all boxes in the circuit."""
        self._update_box_references()
    
    @property
    def size(self) -> int:
        """Return the number of boxes in this circuit."""
        return len(self._boxes)
    
    @property
    def boxes(self) -> List[JunctionBox]:
        """Return read-only view of boxes."""
        return self._boxes.copy()
    
    def __iter__(self) -> Iterator[JunctionBox]:
        """Allow iteration over boxes in the circuit."""
        return iter(self._boxes)
    
    def add_box(self, box: JunctionBox) -> None:
        """Add a junction box to this circuit."""
        if box not in self._boxes:
            self._boxes.append(box)
            box.circuit_id = self.circuit_id
    
    def remove_box(self, box: JunctionBox) -> None:
        """Remove a junction box from this circuit."""
        if box in self._boxes:
            self._boxes.remove(box)
            box.circuit_id = None
    
    def merge_with(self, other: 'Circuit') -> None:
        """Merge another circuit into this one."""
        for box in list(other._boxes):
            other.remove_box(box)
            self.add_box(box)
    
    def _update_box_references(self) -> None:
        """Ensure all boxes reference this circuit ID."""
        for box in self._boxes:
            box.circuit_id = self.circuit_id
    
    def __repr__(self) -> str:
        return f"Circuit(id={self.circuit_id}, size={self.size})"

In [25]:
@dataclass
class Grid:
    """Manages junction boxes, connections, and circuits."""
    junction_boxes: List[JunctionBox]
    connections: List[Connection]
    _circuits: List[Circuit] = field(default_factory=list, repr=False)
    _circuit_map: dict = field(default_factory=dict, init=False, repr=False)
    _next_circuit_id: int = field(default=0, init=False, repr=False)
    
    @property
    def circuits(self) -> List[Circuit]:
        """Return read-only view of circuits."""
        return self._circuits.copy()
    
    @property
    def total_boxes(self) -> int:
        """Return the total number of junction boxes."""
        return len(self.junction_boxes)
    
    @property
    def is_complete(self) -> bool:
        """Check if all junction boxes are assigned to circuits."""
        total_boxes_in_circuits = sum(circuit.size for circuit in self._circuits)
        return total_boxes_in_circuits == self.total_boxes
    
    def get_circuit_by_id(self, circuit_id: Optional[int]) -> Optional[Circuit]:
        """Retrieve a circuit by its ID using O(1) lookup."""
        if circuit_id is None:
            return None
        return self._circuit_map.get(circuit_id)
    
    def activate_connection(self, connection: Connection) -> None:
        """
        Activate a connection between two boxes, updating circuits accordingly.
        
        Rules:
        - If neither box is in a circuit: create a new circuit
        - If one box is in a circuit: add the other to that circuit
        - If both boxes are in different circuits: merge the circuits
        """
        circuit1 = self.get_circuit_by_id(connection.box1.circuit_id)
        circuit2 = self.get_circuit_by_id(connection.box2.circuit_id)
        
        if circuit1 is None and circuit2 is None:
            self._create_new_circuit(connection.box1, connection.box2)
        elif circuit1 is None and circuit2 is not None:
            circuit2.add_box(connection.box1)
        elif circuit1 is not None and circuit2 is None:
            circuit1.add_box(connection.box2)
        elif circuit1 is not None and circuit2 is not None:
            if circuit1.circuit_id != circuit2.circuit_id:
                circuit1.merge_with(circuit2)
                self._remove_circuit(circuit2)
    
    def activate_until_complete(self) -> Optional[Connection]:
        """
        Activate connections until grid is complete.
        Returns the last activated connection.
        """
        for connection in self.connections:
            self.activate_connection(connection)
            if self.is_complete:
                return connection
        return None
    
    def activate_n_shortest(self, n: int) -> None:
        """Activate the n shortest connections."""
        self.sort_connections_by_distance()
        for connection in self.connections[:n]:
            self.activate_connection(connection)
    
    def _create_new_circuit(self, box1: JunctionBox, box2: JunctionBox) -> Circuit:
        """Create a new circuit containing the two boxes."""
        circuit = Circuit(circuit_id=self._next_circuit_id, _boxes=[box1, box2])
        self._circuit_map[self._next_circuit_id] = circuit
        self._next_circuit_id += 1
        self._circuits.append(circuit)
        return circuit
    
    def _remove_circuit(self, circuit: Circuit) -> None:
        """Remove a circuit from the grid."""
        self._circuits.remove(circuit)
        del self._circuit_map[circuit.circuit_id]
    
    def sort_connections_by_distance(self) -> None:
        """Sort all connections by distance (shortest first)."""
        self.connections.sort(key=lambda c: c.distance)
    
    def get_largest_circuits(self, n: int = 3) -> List[Circuit]:
        """Return the n largest circuits by box count."""
        return sorted(self._circuits, key=lambda c: c.size, reverse=True)[:n]
    
    def __repr__(self) -> str:
        return (f"Grid(boxes={self.total_boxes}, "
                f"connections={len(self.connections)}, "
                f"circuits={len(self._circuits)})")

In [26]:
"""Parse input data and create junction boxes."""
junction_boxes = [JunctionBox.from_string(line) for line in data]

print(f"Created {len(junction_boxes)} junction boxes")
print(f"Sample boxes: {junction_boxes[:3]}")

Created 1000 junction boxes
Sample boxes: [JunctionBox(27558, 61383, 12726), JunctionBox(15513, 81970, 25554), JunctionBox(24379, 89821, 82524)]


In [27]:
"""Generate all possible connections between junction boxes."""
connections = Connection.create_all_connections(junction_boxes)

print(f"Total connections: {len(connections)}")
max_connection = max(connections, key=lambda c: c.distance)
print(f"Max distance: {max_connection.distance:.2f}")
print(f"\nSample connections:")
for conn in connections[:3]:
    print(f"  {conn}")

Total connections: 499500
Max distance: 161528.59

Sample connections:
  Connection(JunctionBox(27558, 61383, 12726) <-> JunctionBox(15513, 81970, 25554), dist=27082.54)
  Connection(JunctionBox(27558, 61383, 12726) <-> JunctionBox(24379, 89821, 82524), dist=75435.98)
  Connection(JunctionBox(27558, 61383, 12726) <-> JunctionBox(42987, 15460, 38773), dist=55003.84)


In [28]:
"""Helper function to create a fresh grid instance."""
from copy import deepcopy

def create_grid() -> Grid:
    """Create a new grid with deep-copied boxes and connections."""
    grid_boxes = deepcopy(junction_boxes)
    grid_connections = deepcopy(connections)
    return Grid(junction_boxes=grid_boxes, connections=grid_connections)

## Part 1: Activate shortest 1000 connections

Find the product of the sizes of the three largest circuits after activating the 1000 shortest connections.

In [29]:
"""Create a grid and activate the shortest 1000 connections."""
from math import prod

grid = create_grid()
grid.activate_n_shortest(1000)

print(f"Grid state: {grid}")
print(f"Largest circuits: {grid.get_largest_circuits()}")

Grid state: Grid(boxes=1000, connections=499500, circuits=136)
Largest circuits: [Circuit(id=159, size=42), Circuit(id=164, size=38), Circuit(id=134, size=33)]


In [30]:
"""Calculate product of the three largest circuit sizes."""
largest_circuits = grid.get_largest_circuits(3)
circuit_sizes = [circuit.size for circuit in largest_circuits]
result = prod(circuit_sizes)

print(f"Three largest circuit sizes: {circuit_sizes}")
print(f"Product: {result}")
result

Three largest circuit sizes: [42, 38, 33]
Product: 52668


52668

## Part 2: Complete the grid

Continue activating connections until all junction boxes are in circuits, then calculate the product of the X coordinates of the last activated connection.

In [31]:
"""Activate connections until grid is complete."""
grid2 = create_grid()
grid2.sort_connections_by_distance()

last_activated_connection = grid2.activate_until_complete()

if last_activated_connection:
    result = last_activated_connection.box1.x * last_activated_connection.box2.x
    print(f"Last activated connection: {last_activated_connection}")
    print(f"Product of X coordinates: {result}")
else:
    print("Grid could not be completed")
    result = None

result

Last activated connection: Connection(JunctionBox(37565, 7239, 99565, circuit=115) <-> JunctionBox(39240, 1604, 85100, circuit=115), dist=15613.94)
Product of X coordinates: 1474050600


1474050600